In [1]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Generator, List
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, AIMessageChunk
from langgraph.func import entrypoint


In [2]:
in_memory_store = InMemorySaver()

streaming_handler = StreamingStdOutCallbackHandler()

llm = ChatOllama(
    model="gemma3n",
)

class LLMState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

class LLMToken(TypedDict):
    token: Annotated[str, "The current token being processed"]

In [3]:
def call_model(state: LLMState) -> Generator[LLMState, LLMToken, None]:
    """
    Generates a response from the language model based on the provided state.

    Args:
        state (LLMState): A dictionary containing a list of messages to be processed by the language model.

    Yields:
        LLMToken: The current token being processed from the model's stream.
        LLMState: The final list of AI messages after processing the input state.

    Handles exceptions during streaming and logs any streaming errors.
    """
    output_messages: List[BaseMessage] = []
    messages_for_llm = " ".join([msg.content for msg in state["messages"]])
    full_response = ""
    try:
        for token_chunk in llm.stream(messages_for_llm):
            token_content = token_chunk.content
            full_response += token_content
            yield {"token": token_content}
        ai_message = AIMessage(content=full_response)
        output_messages.append(ai_message)
        yield {"messages": output_messages}
    except Exception as e:
        print("Streaming error:", e)
        yield "[ERROR] Stream interrupted\n"

In [4]:
builder = StateGraph(LLMState)
builder.add_node(call_model)
builder.set_entry_point("call_model")
builder.set_finish_point("call_model")

graph = builder.compile(checkpointer=in_memory_store)

In [5]:
def run_graph_stream(user_input: str):
    try:
        input_message = HumanMessage(content=user_input)
        initial_state = {"messages": [input_message]}

        for tmp_stream_obj in graph.stream(initial_state, stream_mode=["updates", "messages", "custom"], config={"configurable": {"thread_id": "1"}}):
            stream_key, stream_value = tmp_stream_obj
            if stream_key == "messages":
                message = stream_value[0]
                if isinstance(message, AIMessageChunk):
                    yield message.content
    except Exception as e:
        print("Graph execution error:", e)
        yield "[ERROR] Graph execution failed\n"

In [6]:
def run_graph_invoke(user_input: str) -> str:
    """
    Runs the graph to get the complete response at once (invoke-like behavior).

    Args:
        user_input (str): The user's input message.

    Returns:
        str: The complete AI response.
    """
    try:
        input_message = HumanMessage(content=user_input)
        initial_state = {"messages": [input_message]}

        # The 'invoke' method returns the final state
        final_state = graph.invoke(initial_state, config={"configurable": {"thread_id": "1"}})

        # Extract the content of the last AI message
        if final_state and "messages" in final_state:
            ai_messages = [msg for msg in final_state["messages"] if isinstance(msg, AIMessage)]
            if ai_messages:
                # The last AIMessage should contain the full response
                return ai_messages[-1].content
        return "[ERROR] No AI response found."
    except Exception as e:
        print("Graph invoke execution error:", e)
        return "[ERROR] Graph invoke execution failed\n"

In [8]:
for i in run_graph_stream("Hello, how are you?"):
    print(i, end="", flush=True)

Streaming error: [WinError 10061] No connection could be made because the target machine actively refused it
Graph execution error: Expected dict, got [ERROR] Stream interrupted

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE
[ERROR] Graph execution failed


In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

import requests
import json

def get_resume_details_from_llm(resume_text):
    prompt = (
        "Extract the following details from this resume and return as JSON: "
        "name, email, phone, education (list), experience (list), skills (list), certifications (list), summary. "
        "Resume:\n" + resume_text
    )
    result = run_graph_invoke(prompt)
    if not result:
        print("No result returned from LLM.")
        return None
    # Attempt to parse the JSON from the result
    try:
        json_start = result.find("{")
        json_end = result.rfind("}") + 1
        json_str = result[json_start:json_end]
        return json.loads(json_str)
    except Exception as e:
        print("Error parsing JSON:", e)
        print("Raw LLM output:", result)
        return None

# Cell 4: Example usage
pdf_path = r"C:\Users\jigne\Downloads\Copy of 190210107071_Resume.pdf"  # Change to your PDF file path
resume_text = extract_text_from_pdf(pdf_path)
print("Extracted Resume Text:\n", resume_text)

Extracted Resume Text:
  
 
Jignesh
 
Shiyal
 
Technophile,
 
Fast
 
Learner ,
 
Linux
 
Geek,
 
Enthusiasts
 
 
learning
 
new
 
skills
 
and
 
make
 
new
 
experience.I
 
am
 
enthusiastic
 
in
 
many
 
 
ﬁelds
 
like
 
AI,
 
Blockchain
 
And
 
many
 
more
 
things.
 
EXPERIENCE
 
Cybercom
 
Creation
 
(Aug
 
2023
 
-
 
Present)
 
●
 
Position
 
:
 
Software
 
Developer
 
Cybercom
 
Creation
 
(Feb
 
2023
 
-
 
Jul
 
2023)
 
●
 
Position
 
:
 
Intern
 
●
 
Internship
 
Project:
 
Event
 
management
 
system
 
automates
 
planning,
 
organizing,
 
and
 
managing
 
events
 
with
 
features
 
such
 
as
 
registration
 
and
 
budgeting
 
PROJECTS
 
TableGPT
 
—
 
W rite
 
SQL
 
query
 
using
 
English
 
language
 
●
 
Fine-tune
 
LLM
 
(LLAMA-2
 
7b)
 
for
 
text
 
to
 
sql
 
task
 
Product
 
M atching
 
—
 
ﬁnd
 
sim ilar
 
product
 
●
 
Using
 
CNN
 
model
 
(ImageNet-v3),
 
create
 
deep
 
learning
 
pipeline
 
For
 
identiﬁed
 
similar
 
product
 
 
EDUCATION
 
 
Class/Sem
 
Board/Un

In [18]:
resume_json = get_resume_details_from_llm(resume_text)
print(resume_json)

{'name': 'JiGnesh ShiyaL', 'email': 'jignesh.shiyal1122@gmail.com', 'phone': '9106762544', 'education': [{'class': 'B.E (CGPA)', 'sem': 'Not specified', 'board': 'GEC (Bhavnagar)', 'university': 'Not specified', 'percentage': '8.70', 'cGPA': None}, {'class': 'H.S.C', 'sem': 'Not specified', 'board': 'GSEB', 'university': 'Not specified', 'percentage': '71.33', 'cGPA': None}, {'class': 'S.S.C', 'sem': 'Not specified', 'board': 'GSEB', 'university': 'Not specified', 'percentage': '86.16', 'cGPA': None}], 'experience': [{'company': 'Cybercom Creation', 'position': 'Software Developer', 'start_date': 'Aug 2023', 'end_date': 'Present'}, {'company': 'Cybercom Creation', 'position': 'Intern', 'start_date': 'Feb 2023', 'end_date': 'Jul 2023', 'project': 'Event management system automates planning, organizing, and managing events with features such as registration and budgeting'}], 'skills': ['Deep Learning (7/10)', 'Machine Learning (7/10)', 'Python (8/10)', 'Django (7/10)', 'LLM (6/10)', 'AI'

In [19]:
resume_json

{'name': 'JiGnesh ShiyaL',
 'email': 'jignesh.shiyal1122@gmail.com',
 'phone': '9106762544',
 'education': [{'class': 'B.E (CGPA)',
   'sem': 'Not specified',
   'board': 'GEC (Bhavnagar)',
   'university': 'Not specified',
   'percentage': '8.70',
   'cGPA': None},
  {'class': 'H.S.C',
   'sem': 'Not specified',
   'board': 'GSEB',
   'university': 'Not specified',
   'percentage': '71.33',
   'cGPA': None},
  {'class': 'S.S.C',
   'sem': 'Not specified',
   'board': 'GSEB',
   'university': 'Not specified',
   'percentage': '86.16',
   'cGPA': None}],
 'experience': [{'company': 'Cybercom Creation',
   'position': 'Software Developer',
   'start_date': 'Aug 2023',
   'end_date': 'Present'},
  {'company': 'Cybercom Creation',
   'position': 'Intern',
   'start_date': 'Feb 2023',
   'end_date': 'Jul 2023',
   'project': 'Event management system automates planning, organizing, and managing events with features such as registration and budgeting'}],
 'skills': ['Deep Learning (7/10)',
  

In [22]:
import requests
from bs4 import BeautifulSoup

def get_job_post_text(job_input):
    """
    Accepts either a job description text or a URL.
    If a URL is provided, fetches and extracts visible text from the page.
    """
    if job_input.startswith("http://") or job_input.startswith("https://"):
        try:
            resp = requests.get(job_input, timeout=15)
            soup = BeautifulSoup(resp.text, "html.parser")
            # Remove script/style and get visible text
            for tag in soup(["script", "style", "noscript"]):
                tag.decompose()
            text = " ".join(soup.stripped_strings)
            return text
        except Exception as e:
            print("Error fetching job post:", e)
            return ""
    else:
        return job_input

def extract_trending_keywords_from_job(job_input):
    """
    Extracts trending/important keywords from a job post (text or link).
    """
    job_text = get_job_post_text(job_input)
    if not job_text:
        print("No job post text found.")
        return []
    prompt = (
        "Extract the top 10 trending and important keywords (skills, technologies, certifications, roles) "
        "from the following job description. Return as a JSON list of strings.\n\n"
        "Job Description:\n" + job_text
    )
    result = run_graph_invoke(prompt)
    try:
        json_start = result.find("[")
        json_end = result.rfind("]") + 1
        json_str = result[json_start:json_end]
        return json.loads(json_str)
    except Exception as e:
        print("Error parsing keywords JSON:", e)
        print("Raw LLM output:", result)
        return []

In [24]:
# Example usage:
job_input = "https://in.linkedin.com/jobs/data-science-jobs-ahmedabad?currentJobId=4259410101&position=1&pageNum=0"  # or paste job description text here
keywords = extract_trending_keywords_from_job(job_input)
print("Trending Keywords:", keywords)

Trending Keywords: ['Data Scientist', 'AI/ML Engineer', 'Machine Learning Engineer', 'Artificial Intelligence (AI)', 'Data Analyst', 'Data Engineering', 'Business Analyst', 'Data Research', 'Software Developer', 'Data Management', 'AI Consultant', 'Data Analysis', 'Deep Learning', 'Natural Language Processing (NLP)', 'Computer Vision', 'R Programming', 'Python Programming', 'TensorFlow', 'PyTorch', 'Cloud Computing', 'Deep Learning', 'Data Mining', 'Predictive Modeling', 'Statistical Modeling', 'Big Data', 'Spark', 'Hadoop', 'Data Visualization', 'SQL', 'NoSQL', 'Cloud', 'LLM', 'LLM Models']
